In [10]:
import json

cat_json = {}
cat_txt = []
with open("grouped_categories.json") as f:
    cat_json = json.load(f)

with open("gm_places_categories.txt") as f:
    for l in f:
        cat_txt.append(l.split("\n")[0]) 

In [11]:
distinct_from_json = set()
distinct_txt = set(cat_txt)

for k, v in cat_json.items():
    for el in v:
        distinct_from_json.add(el)

not_used = list(distinct_txt.difference(distinct_from_json))
not_used

['accounting']

In [13]:
texts = ["""I really like visiting museums and exploring culture. 
I want to visit some cultural events. I prefer cheap accomodation.""",
        """I love going to the night clubs. I hope to go to parties in the evenings. During the day I want to fine dine
        and visit some bars."""]

from my_nlp_module.preprocessing import preprocess_document, PrepOption

options = [PrepOption.NUMBERS, PrepOption.LOWERCASE, PrepOption.RUBBISH, PrepOption.INTERPUNCTION, PrepOption.TOKENIZE_SENTENCE]

documents_preprocessed = []

for doc in texts:
    p = preprocess_document(doc, options)
    print(p)


['i really like visiting museums and exploring culture', 'i want to visit some cultural events', 'i prefer cheap accomodation']
['i love going to the night clubs', 'i hope to go to parties in the evenings', 'during the day i want to fine dine and visit some bars']


In [26]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome',
             "Test sentence"]

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)
print(cosine_scores)
cosine_scores = cosine_scores.numpy()
cosine_scores

tensor([[ 0.2838,  0.1310, -0.0029],
        [ 0.2277, -0.0327, -0.0136],
        [ 0.0543, -0.0502,  0.8939],
        [-0.0114,  0.0125,  0.0397]])


array([[ 0.28375503,  0.13100722, -0.00286157],
       [ 0.22769953, -0.03272609, -0.01360959],
       [ 0.05429638, -0.0501904 ,  0.89390373],
       [-0.01136757,  0.0125467 ,  0.03974972]], dtype=float32)

In [15]:
print(cosine_scores)

tensor([[ 0.2838,  0.1310, -0.0029],
        [ 0.2277, -0.0327, -0.0136],
        [ 0.0543, -0.0502,  0.8939],
        [-0.0114,  0.0125,  0.0397]])


In [63]:
from my_nlp_module.preprocessing import preprocess_document, PrepOption
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
analyzer = SentimentIntensityAnalyzer()

texts = ["""I really like visiting museums and exploring culture. 
I want to visit some cultural events. I prefer expensive accomodation.""",
        """I love going to the night clubs. I hope to go to parties in the evenings. During the day I want to fine dine
        and visit some bars.""",
        """I hate going to the night clubs. I want to visit some cultural things."""]

templates = ["museum history buildings culture tradition", "party nigh club clubs drinking alcohol", 
             "restaurant food expensive tasty", "cheap accomodation hostel"]

options1 = [PrepOption.TOKENIZE_SENTENCE]
options2 = [PrepOption.NUMBERS, PrepOption.LOWERCASE, PrepOption.RUBBISH, PrepOption.INTERPUNCTION]


for doc in texts:
    sentences = preprocess_document(doc, options1)
    n_topics = 4
    doc_sim = np.zeros(shape=(n_topics))
    normalization = [0 for i in range(n_topics)]
    for s in sentences:
        vs = analyzer.polarity_scores(s)
        prep_sent = preprocess_document(s, options2)
        embeddings1 = model.encode(prep_sent, convert_to_tensor=True)
        embeddings2 = model.encode(templates, convert_to_tensor=True)
        sim = util.cos_sim(embeddings1, embeddings2).numpy().squeeze()
        for i in range(sim.shape[0]):
            sim[i] = sim[i] if np.abs(sim[i])>0.2 else 0
        norm_idx = np.where(sim!=0)[0]
        for i in norm_idx:
            normalization[i] += 1
        print(s, sim, sim*(1-vs["neg"]))
        sim = sim*(1-vs["neg"])
        doc_sim += sim
    for i, el in enumerate(normalization):
        if el > 0:
            doc_sim[i] /= el
    #doc_sim /= len(sentences)
    #doc_sim = (doc_sim + 1)/2
    print(f"{doc}\n{doc_sim}\n\n")


I really like visiting museums and exploring culture. [0.48005053 0.         0.         0.        ] [0.48005053 0.         0.         0.        ]
I want to visit some cultural events. [0.4649859  0.230849   0.22161081 0.        ] [0.4649859  0.230849   0.22161081 0.        ]
I prefer expensive accomodation. [0.         0.         0.36597836 0.72362435] [0.         0.         0.36597836 0.72362435]
I really like visiting museums and exploring culture. 
I want to visit some cultural events. I prefer expensive accomodation.
[0.47251822 0.230849   0.29379459 0.72362435]


I love going to the night clubs. [0.       0.518558 0.       0.      ] [0.       0.518558 0.       0.      ]
I hope to go to parties in the evenings. [0.         0.45091712 0.         0.        ] [0.         0.45091712 0.         0.        ]
During the day I want to fine dine and visit some bars. [0.         0.388821   0.3166833  0.26036456] [0.         0.388821   0.3166833  0.26036456]
I love going to the night clubs. I 

In [45]:
documents_preprocessed = []

for doc in texts:
    p = preprocess_document(doc, options1)
    documents_preprocessed.append(p)

embeddings1 = model.encode(documents_preprocessed, convert_to_tensor=True)
embeddings2 = model.encode(templates, convert_to_tensor=True)
sim = util.cos_sim(embeddings1, embeddings2)
(sim+1)/2

tensor([[0.7479, 0.5993, 0.5858, 0.5853],
        [0.5484, 0.7651, 0.5501, 0.5402],
        [0.6818, 0.7333, 0.6194, 0.6339]])